In [1]:
import etl
import vis

In [2]:
etl.unpack_interactive()

Enter the name of the raw data folder:  data
Enter the name of the zip file (include .zip):  test_data.zip


In [12]:

# tests on alfred.vis

import pandas as pd

directory = 'test_data_after_sorted/'

def test_build_bus_df_1():
    '''
    Tests to determine that the return is a Pandas Dataframe
    '''
    bus_df = build_bus_df(directory, 'bus_0039141908140233500710/', 'Current')
    assert isinstance(bus_df, pd.DataFrame), 'Pandas DataFrame not returned'
    return

def test_build_module_df_1():
    '''
    Tests to determine that the return is a Pandas Dataframe
    '''
    module_df = build_module_df(directory, 'bus_0039141908140233500710/', 1)
    assert isinstance(module_df, pd.DataFrame), 'Pandas DataFrame not returned'
    return


directory = 'test_data_after_sorted/'

def test_find_replaced_modules_1():
    '''
    Tests to determine that the returned object is a dictionary
    '''
    replaced_dict = find_replaced_modules(directory)
    assert isinstance(replaced_dict, dict), 'Dictionary not returned'
    return

def test_find_replaced_modules_2():
    '''
    Tests to determine dictionary keys are of type strings.
    '''
    replaced_dict = find_replaced_modules(directory)
    for key in replaced_dict.keys():
        assert isinstance(key, str), 'Bus names are not strings.'
    return

def test_find_replaced_modules_3():
    '''
    Tests to determine that the dictionary values are lists.
    '''
    replaced_dict = find_replaced_modules(directory)
    for value in replaced_dict.values():
        assert isinstance(value, list), 'Returned values are not in a list format.'
    return

def test_find_replaced_modules_4():
    '''
    Tests to determine that the returned dictionary values' contents are of type string
    '''
    replaced_dict = find_replaced_modules(directory)
    for value in replaced_dict.values():
        for module in value:
            assert isinstance(module, str), 'Returned serials are not of type string.'
    return

def test_swapped_mod_dataframes_1():
    '''
    Tests to determine that a list is returned.
    '''
    df_swap = swapped_mod_dataframes(directory, 'g2--..-.-.-.-', 'cell voltages')
    assert isinstance(df_swap, list), "A list of dataframes is not returned"
    return

def test_swapped_mod_dataframes_2():
    '''
    Tests to determine that a list is returned.
    '''
    df_swap = swapped_mod_dataframes(directory, 'g2--..-.-.-.-', 'cell voltages')
    for df in df_swap:
        assert isinstance(df, pd.DataFrame), "Not all contents of list are Pandas Dataframes"
    return

import numpy as np
import os
from os import listdir

directory = 'test_data_after_sorted/'

def test_count_bus_file_1():
    '''
    Test to determine that returned bus folders of test data is 2.
    '''
    bus_count = count_bus_file(directory)
    assert bus_count == 2, "Number of buses in folder should be 2"
    return

def test_filter_false_module_1():
    '''
    Test the function should return an array type
    '''
    return_type = filter_false_module(directory)
    assert isinstance(return_type,np.ndarray),'The return type is not an array'
    return

def test_filter_false_module_2():
    '''
    Test the function should return a list type and each element should be string
    '''
    return_list = filter_false_module(directory)
    for element in return_list:
        assert isinstance(element,str),'Each element in this list should be string.'
    return

def test_move_false_bus_1():
    '''
    Test each folder that get relocated should have at least two csv files.
    '''
    for file in listdir(directory):
        if file.startswith('bus'):
            directory_path = os.path.join(directory, file)
            each_bus = listdir(directory_path)
            assert len(each_bus) >= 2, 'Each bus file should contain at least two csv files.'
        else:
            pass
    return

def test_move_false_bus_2():
    '''
    Test each folder that get relocated should have be all csv type files.
    '''
    for file in listdir(directory):
        directory_path = os.path.join(directory, file)
        each_bus = listdir(directory_path)
        for element in each_bus:
            assert element.endswith('.csv'), 'There is a non csv file.'
    return


def test_compare_file_mods_1():
    '''
    Tests that function returns a dictionary
    '''
    return_type = compare_file_mods(directory)
    assert isinstance(return_type,dict), 'The returned type is not a dictionary'
    return


def test_compare_file_mods_2():
    '''
    Tests that dictionary returns a pandas dataframe when type in attribute name, such as "bus_1"
    '''
    test_folder = ['bus_1','bus_2']
    dic = compare_file_mods(directory)
    for folder_name in test_folder:
        dic_key = dic[folder_name]
        assert isinstance(dic_key,pd.DataFrame), 'The returned type is not a pandas dataframe'
    return


def test_compare_file_mods_3():
    '''
    Tests that returned dataframe has 16 index, from "Module 1" to "Module 16"
    '''
    test_folder = ['bus_1', 'bus_2']
    dic = compare_file_mods(directory)
    for folder_name in test_folder:
        dic_key = dic[folder_name]
        index_len = dic_key.index
        assert len(index_len) == 16, 'The dataframe is not in complete form, module number is not 16'
    return

In [13]:
unittest.main(argv=[''], verbosity=2, exit=False)


----------------------------------------------------------------------
Ran 0 tests in 0.000s

OK


In [4]:
import os
import shutil
import tempfile
import unittest
import csv
from etl import group_files

class TestGroupFiles(unittest.TestCase):

    def setUp(self):
        # Create a temporary directory
        self.test_dir = tempfile.mkdtemp()
        self.test_data_dir = os.path.join(self.test_dir, 'sorted_test_data/all_buses')
        os.makedirs(self.test_data_dir)

        # Add test CSV file
        self.test_file_name = '14F0154_ProfileData_20171116125343(2).csv'
        self.test_file_path = os.path.join(self.test_data_dir, self.test_file_name)
        
        # Write test data to the CSV file
        serial_number = '0039141908140233500710'
        keyword = 'Mfg Data (ASCII) '
        with open(self.test_file_path, 'w', newline='') as f:
            writer = csv.writer(f)
            # Add a row with the 'Mfg Data (ASCII)' keyword and serial number
            writer.writerow(['Some other data', f'{keyword}{serial_number}'])
            for _ in range(15):  # Add enough serial numbers to trigger directory creation
                writer.writerow(['Some other data', f'{keyword}{serial_number}'])

    def tearDown(self):
        # Remove the temporary directory after the test
        shutil.rmtree(self.test_dir)

    def test_group_files(self):
        # Call the group_files function
        group_files(self.test_data_dir)

        # Define the expected directory and file path
        expected_dir_name = 'bus_0039141908140233500710'
        expected_dir = os.path.join(self.test_data_dir, expected_dir_name)
        expected_file = os.path.join(expected_dir, self.test_file_name)
        
        # Check if the expected directory exists
        self.assertTrue(os.path.isdir(expected_dir), f"Expected directory {expected_dir} does not exist.")
        
        # Check if the expected file exists in the new directory
        self.assertTrue(os.path.isfile(expected_file), f"Expected file {expected_file} does not exist in {expected_dir}.")

if __name__ == '__main__':
    suite = unittest.TestLoader().loadTestsFromTestCase(TestGroupFiles)
    unittest.TextTestRunner().run(suite)


F
FAIL: test_group_files (__main__.TestGroupFiles.test_group_files)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/tmp/ipykernel_122037/1270366558.py", line 44, in test_group_files
    self.assertTrue(os.path.isdir(expected_dir), f"Expected directory {expected_dir} does not exist.")
AssertionError: False is not true : Expected directory /tmp/tmpd0i32ljl/sorted_test_data/all_buses/bus_0039141908140233500710 does not exist.

----------------------------------------------------------------------
Ran 1 test in 0.005s

FAILED (failures=1)


In [12]:
import os
import shutil
import tempfile
import unittest
import csv
import pkg_resources
from etl import group_files

class TestGroupFiles(unittest.TestCase):

    def setUp(self):
        # Create a temporary directory
        self.test_dir = tempfile.mkdtemp()
        self.test_data_dir = os.path.join(self.test_dir, 'all_buses')
        os.makedirs(self.test_data_dir)

        # Path to the actual test data in the repository
        original_test_data_dir = pkg_resources.resource_filename('alfred/tests', 'unsorted_test_data/all_buses')
        
        # Copy the test data from the repository to the temporary directory
        for filename in os.listdir(original_test_data_dir):
            shutil.copy(os.path.join(original_test_data_dir, filename), self.test_data_dir)

        # Print statements for debugging
        print(f"Setup: Created test directory at {self.test_data_dir}")
        print(f"Setup: Copied test data from {original_test_data_dir}")

        # Print the content of the CSV files in the test directory
        for filename in os.listdir(self.test_data_dir):
            file_path = os.path.join(self.test_data_dir, filename)
            with open(file_path, 'r') as f:
                content = f.readlines()
            print(f"Setup: Content of the file {filename}:\n{''.join(content)}")

    def tearDown(self):
        # Print statements for debugging
        print(f"TearDown: Removing test directory {self.test_dir}")

        # Remove the temporary directory after the test
        shutil.rmtree(self.test_dir)

    def test_group_files(self):
        # Call the group_files function
        group_files(self.test_data_dir)

        # Define the expected directory and file path
        expected_dir_name = 'bus_0039141908140233500710'
        expected_dir = os.path.join(self.test_data_dir, expected_dir_name)
        expected_file = os.path.join(expected_dir, '14F0154_ProfileData_20171116125343(2).csv')
        
        # Print statements for debugging
        print(f"Test: Expected directory: {expected_dir}")
        print(f"Test: Expected file: {expected_file}")

        # Check if the expected directory exists
        dir_exists = os.path.isdir(expected_dir)
        print(f"Test: Directory exists: {dir_exists}")
        self.assertTrue(dir_exists, f"Expected directory {expected_dir} does not exist.")
        
        # Check if the expected file exists in the new directory
        file_exists = os.path.isfile(expected_file)
        print(f"Test: File exists: {file_exists}")
        self.assertTrue(file_exists, f"Expected file {expected_file} does not exist in {expected_dir}")

if __name__ == '__main__':
    suite = unittest.TestLoader().loadTestsFromTestCase(TestGroupFiles)
    unittest.TextTestRunner().run(suite)


E
ERROR: test_group_files (__main__.TestGroupFiles.test_group_files)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/home/clgould99/miniconda3/lib/python3.11/site-packages/pkg_resources/__init__.py", line 402, in get_provider
    module = sys.modules[moduleOrReq]
             ~~~~~~~~~~~^^^^^^^^^^^^^
KeyError: 'alfred/tests'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/tmp/ipykernel_122037/3865055541.py", line 18, in setUp
    original_test_data_dir = pkg_resources.resource_filename('alfred/tests', 'unsorted_test_data/all_buses')
                             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/clgould99/miniconda3/lib/python3.11/site-packages/pkg_resources/__init__.py", line 1213, in resource_filename
    return get_provider(package_or_requirement).get_resource_filename(
           ^^^^^^^^^^^^^

In [1]:
sorted_directory = 'alfred/sorted_test_data/'
unsorted_directory = 'alfred/unsorted_test_data/'
raw_directory = 'alfred/unzipped_test_data/'

In [4]:
import unittest
import os
import tempfile
import zipfile
import etl
import vis
from vis import build_bus_df

class TestVis1(unittest.TestCase):
    def test_build_bus_df_1(directory):
        '''
        Tests to determine that the return is a Pandas Dataframe
        '''
        bus_df = build_bus_df(directory, 'bus_0039141908140233500710/', 'Current')
        assert isinstance(bus_df, pd.DataFrame), 'Pandas DataFrame not returned'
        return
if __name__ == '__main__':
    suite = unittest.TestLoader().loadTestsFromTestCase(TestVis1)
    unittest.TextTestRunner().run(suite)

E
ERROR: test_build_bus_df_1 (__main__.TestVis1.test_build_bus_df_1)
Tests to determine that the return is a Pandas Dataframe
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/tmp/ipykernel_24734/655553593.py", line 14, in test_build_bus_df_1
    bus_df = build_bus_df(directory, 'bus_0039141908140233500710/', 'Current')
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/clgould99/TheBattmobile/Alfred/alfred/vis.py", line 84, in build_bus_df
    bus_dates = sort_bus_by_date(directory, bus_num)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/clgould99/TheBattmobile/Alfred/alfred/etl.py", line 196, in sort_bus_by_date
    bus_directory = directory + bus_num
                    ~~~~~~~~~~^~~~~~~~~
TypeError: unsupported operand type(s) for +: 'TestVis1' and 'str'

----------------------------------------------------------------------
Ran 1 test in 0.003s

FAILED (er

In [2]:
import os

os. getcwd()


'/home/clgould99/TheBattmobile/Alfred/alfred'

In [6]:
import vis
import etl

In [10]:
directory = etl.find_directory() + 'sorted_test_data/vis_buses/'

vis.build_bus_df(directory, 'bus_0039142903b40233500710/', 'Voltage')

,< 450.0,450,460,470,480,490,500,510,520,530,...,670,680,690,700,710,720,730,740,>= 750.0,TOTAL
0,0,0,0,0,0,0,2,0,33,439,...,301690,194716,106311,18151,1063,4,0,0,0,11730296
1,27,0,0,0,0,0,2,0,33,440,...,305173,196624,107354,18399,1099,5,0,1,7,11912854
2,27,0,0,0,0,0,2,0,70,646,...,460645,297335,153481,23650,1385,5,0,1,7,17233181


In [6]:
import unittest
import pandas as pd
import os
import etl
from vis import build_bus_df
# Assuming build_bus_df is imported from the module where it is defined
# from your_module import build_bus_df

class TestBuildBusDF(unittest.TestCase):
    def setUp(self):
        # Setup code here if needed
        self.valid_directory = os.path.join(etl.find_directory(), 'sorted_test_data/vis_buses/')
        self.bus_num = "bus_0039142903b40233500710/"  
        #self.valid_keywords = ["Current", "Voltage", "Power"]

    def test_dataframe_creation_current(self):
        # Test to ensure a DataFrame is created for 'Current'
        keyword = "Current"
        result = build_bus_df(self.valid_directory, self.bus_num, keyword)
        self.assertIsInstance(result, pd.DataFrame, "The result should be a pandas DataFrame for 'Current'")

    def test_dataframe_creation_voltage(self):
        # Test to ensure a DataFrame is created for 'Voltage'
        keyword = "Voltage"
        result = build_bus_df(self.valid_directory, self.bus_num, keyword)
        self.assertIsInstance(result, pd.DataFrame, "The result should be a pandas DataFrame for 'Voltage'")

    def test_dataframe_creation_power(self):
        # Test to ensure a DataFrame is created for 'Power'
        keyword = "Power"
        result = build_bus_df(self.valid_directory, self.bus_num, keyword)
        self.assertIsInstance(result, pd.DataFrame, "The result should be a pandas DataFrame for 'Power'")


    def tearDown(self):
        # Clean up code here if needed
        pass

if __name__ == '__main__':
    suite = unittest.TestLoader().loadTestsFromTestCase(TestBuildBusDF)
    unittest.TextTestRunner().run(suite)



...
----------------------------------------------------------------------
Ran 3 tests in 0.106s

OK


In [24]:
import unittest
import pandas as pd
import os
import etl
from vis import build_module_df
#import alfred
# Assuming build_module_df is imported from the module where it is defined
# from build_data_vis import build_module_df

class TestBuildModuleDF(unittest.TestCase):
    def setUp(self):
        # Setup code here if needed
        self.valid_directory = os.path.join(etl.find_directory(), 'sorted_test_data/vis_buses/')
        self.invalid_directory = os.path.join('alfred/sorted_test_data/sorted_data/incomplete_test/')
        self.bus_num = "bus_0039142903b40233500710/"  
    def test_dataframe_creation_for_valid_numbers(self):
        # Test to ensure a DataFrame is created for each valid number from 1 to 16
        for num in range(1, 17):
            with self.subTest(num=num):
                result = build_module_df(self.valid_directory, self.bus_num, num)
                self.assertIsInstance(result, pd.DataFrame, f"The result should be a pandas DataFrame for module number {num}")

    def test_no_dataframe_creation_for_invalid_data(self):
        # Test to ensure no DataFrame is created for invalid/incomplete data
        num = 1
        try:
            result = build_module_df(self.invalid_directory, self.bus_num, num)
            # Check if the function returns None or handles the case gracefully
            self.assertIsNone(result, "The result should be None for invalid/incomplete data")
        except Exception as e:
            # If an exception is raised, ensure it's the expected behavior
            self.assertIsInstance(e, (FileNotFoundError, pd.errors.EmptyDataError), "Expected an error due to invalid data")
    def test_invalid_number_input(self):
    # Test to ensure no DataFrame is created for invalid module number
        invalid_numbers = [ 17, -1, 'a']
        for num in invalid_numbers:
            with self.subTest(num=num):
                with self.assertRaises((ValueError, TypeError), msg=f"Expected a ValueError or TypeError for invalid module number {num}"):
                    build_module_df(self.valid_directory, self.bus_num, num)

    def tearDown(self):
        # Clean up code here if needed
        pass

if __name__ == '__main__':
    suite = unittest.TestLoader().loadTestsFromTestCase(TestBuildModuleDF)
    unittest.TextTestRunner().run(suite)


...
----------------------------------------------------------------------
Ran 3 tests in 0.539s

OK
